[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colab-samples/blob/main/notebooks/basic_notebook_features/text_cells.ipynb)




In [ ]:
# Install PyCaret from GitHub master branch
!pip install git+https://github.com/pycaret/pycaret.git@master --upgrade -q

In [ ]:
# Load Boston Housing dataset for regression
from sklearn.datasets import fetch_openml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load Boston Housing dataset from OpenML
boston = fetch_openml(name='boston', version=1, as_frame=True, parser='auto')
df = boston.frame

# Rename target column for clarity
df.rename(columns={'MEDV': 'house_price'}, inplace=True)

print(f"✅ Boston Housing Dataset loaded: {df.shape}")
print(f"\nTarget Statistics (House Prices in $1000s):")
print(df['house_price'].describe())
print(f"\nFeatures: {list(df.columns[:-1])}")
df.head()

In [ ]:
from pycaret.regression import *

# Initialize regression environment
reg_exp = setup(
    data=df,
    target='house_price',
    session_id=456,
    normalize=True,
    transformation=True,
    remove_outliers=True,
    polynomial_features=True,
    fold=10,
    verbose=False
)

In [ ]:
# Check if GPU is connected
!nvidia-smi --query-gpu=name --format=csv,noheader

In [ ]:
# Compare all regression models
print("🔍 Comparing regression models...")
top5 = compare_models(n_select=5, sort='R2')
print("\n✅ Top 5 models selected based on R² score")

In [ ]:
# Get the best model
best_model = top5[0]

print("📊 Evaluating the best regression model...")
print(f"Best Model: {best_model.__class__.__name__}")

# Generate multiple evaluation plots
plot_model(best_model, plot='residuals')
plot_model(best_model, plot='error')
plot_model(best_model, plot='feature')
plot_model(best_model, plot='learning')

In [ ]:
# Interactive dashboard for model evaluation
evaluate_model(best_model)

In [ ]:
# Tune hyperparameters for optimal performance
print("⚡ Optimizing hyperparameters...")
tuned_model = tune_model(best_model, n_iter=20, optimize='MAE')

print("\n📈 Performance Metrics:")
print("Tuned Model Results:")

In [ ]:
# Train on full dataset
final_model = finalize_model(tuned_model)

# Generate predictions on test set
predictions = predict_model(final_model)

# Calculate prediction metrics
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(predictions['house_price'], predictions['prediction_label'])
r2 = r2_score(predictions['house_price'], predictions['prediction_label'])

print(f"\n✅ Test Set Performance:")
print(f"   Mean Absolute Error: ${mae:.2f}k")
print(f"   R² Score: {r2:.4f}")
print(f"\n📊 Sample Predictions (Actual vs Predicted):")

predictions[['house_price', 'prediction_label']].head(10)

In [ ]:
# Plot actual vs predicted prices
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(predictions['house_price'], predictions['prediction_label'], alpha=0.6)
plt.plot([predictions['house_price'].min(), predictions['house_price'].max()],
         [predictions['house_price'].min(), predictions['house_price'].max()],
         'r--', lw=2)
plt.xlabel('Actual House Price ($1000s)', fontsize=12)
plt.ylabel('Predicted House Price ($1000s)', fontsize=12)
plt.title('Actual vs Predicted House Prices', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ Perfect predictions would fall on the red diagonal line")

In [ ]:
# Save the final model
model_name = 'boston_housing_regression_model'
save_model(final_model, model_name)

print(f"✅ Model successfully saved as '{model_name}.pkl'")
print(f"📦 Model can be loaded using: loaded_model = load_model('{model_name}')")
print(f"\n🎯 Model Summary:")
print(f"   • Dataset: Boston Housing (506 samples)")
print(f"   • Target: Median house prices")
print(f"   • Best Algorithm: {best_model.__class__.__name__}")
print(f"   • Test MAE: ${mae:.2f}k")
print(f"   • Test R²: {r2:.4f}")